# Chapter 7.4: Self-RAG (자기 평가 기반 검색)

이 노트북은 OSTEP RAG 시스템에 Self-RAG 기법을 적용하여 검색 품질과 응답 신뢰도를 개선하는 방법을 실습합니다.

## 📚 학습 목표
- Self-RAG 개념 이해: 검색 필요성, 관련성, 지원도를 자동 평가
- LLM 기반 다단계 평가 프로세스 구현
- Ollama를 활용한 로컬 LLM 기반 Self-RAG 구현

## 📋 실습 구성
1) 설정 및 하이퍼파라미터 정의
2) 기존 데이터 로드 및 Ollama 설정
3) Self-RAG 개념 설명
4) Self-RAG 함수 구현 (검색 필요성/관련성/지원도 평가)
5) 데모: Self-RAG 전체 프로세스 실행
6) 요약 및 다음 단계

> **핵심 아이디어**: RAG 시스템이 스스로 "검색이 필요한가?", "검색 결과가 관련있는가?", "응답이 근거 기반인가?"를 평가하여 더 신뢰할 수 있는 답변을 생성합니다.


---
## 1️⃣ 설정 및 하이퍼파라미터 정의

이 셀에서는 경로, 모델, 검색 파라미터 등을 정의합니다.

**주요 내용:**
- 기존 산출물 경로 (청크 JSON, FAISS 인덱스)
- Ollama 서버 설정 (Self-RAG용 LLM)
- Self-RAG 파라미터

**실행 결과:**
- 설정값이 출력됩니다.


In [ ]:
# ========================================
# 1️⃣ 설정 / 하이퍼파라미터
# ========================================
import os
import json
import faiss
import numpy as np
import torch
import requests
from pathlib import Path
from typing import List, Dict, Any, Tuple

# 재현성을 위한 시드 설정
SEED = 42
np.random.seed(SEED)

# 디바이스
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 경로 (기존 산출물)
CHUNK_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/chunk/ostep_tok400_ov20.json"
INDEX_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw.index"

# 검색 하이퍼파라미터
TOP_K = 3
EF_SEARCH = 64

# 임베딩 모델
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
NORMALIZE = True

# Ollama 설정 (Self-RAG용 LLM)
OLLAMA_HOST = "http://localhost:11434"
LLM_MODEL = "llama3.1:8b"
TEMPERATURE = 0.2  # 낮은 temperature로 일관된 평가

print(f"🔧 Device: {DEVICE}")
print(f"📁 청크 파일: {CHUNK_FILE}")
print(f"📁 인덱스 파일: {INDEX_FILE}")
print(f"🔎 TOP_K={TOP_K}")
print(f"🧠 임베딩 모델: {EMBED_MODEL}")
print(f"🤖 LLM: {LLM_MODEL} @ {OLLAMA_HOST}")


---
## 2️⃣ 기존 데이터 로드 및 Ollama 설정

Chapter 2-4에서 생성한 청크 JSON과 FAISS 인덱스를 로드하고, Ollama 서버 연결을 확인합니다.

**주요 내용:**
- 청크 데이터 로드
- FAISS 인덱스 로드
- 임베딩 모델 로드
- Ollama 서버 연결 확인

**실행 결과:**
- 데이터 로드 완료 메시지와 Ollama 서버 상태가 출력됩니다.


In [ ]:
# 청크 데이터 로드
print("청크 데이터 로드 중...")
with open(CHUNK_FILE, 'r', encoding='utf-8') as f:
    CHUNKS = json.load(f)
print(f"✓ 청크 로드 완료: {len(CHUNKS)}개")

# 인덱스 로드
print("\n인덱스 로드 중...")
INDEX = faiss.read_index(INDEX_FILE)
print(f"✓ 인덱스 로드 완료: {INDEX.ntotal}개 벡터")

# 임베딩 모델 로드
print(f"\n임베딩 모델 로드 중: {EMBED_MODEL}")
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer(EMBED_MODEL, device="cpu" if DEVICE=="cpu" else DEVICE)
print("✓ 임베딩 모델 로드 완료")

# Ollama 서버 연결 확인
print(f"\nOllama 서버 연결 확인: {OLLAMA_HOST}")
try:
    response = requests.get(f"{OLLAMA_HOST}/api/version", timeout=5)
    if response.status_code == 200:
        print(f"✓ Ollama 서버 연결 성공")
    else:
        print(f"⚠️  Ollama 서버 응답 오류: {response.status_code}")
except Exception as e:
    print(f"⚠️  Ollama 서버 연결 실패: {e}")
    print("   Ollama 서버가 실행 중인지 확인하세요: ollama serve")


---
## 3️⃣ Self-RAG 개념 설명

이 셀에서는 Self-RAG의 핵심 개념을 설명합니다.

**Self-RAG란?**
- **문제**: RAG 시스템이 항상 검색이 필요한지, 검색 결과가 관련있는지, 답변이 근거 기반인지 모름
- **해결책**: 다단계 자기 평가 프로세스
  1. **검색 필요성 평가**: 이 질문에 검색이 필요한가? (Yes/No)
  2. **관련성 평가**: 검색된 문서가 질문과 관련있는가? (Relevant/Irrelevant)
  3. **응답 생성**: 관련 문서 기반으로 답변 생성
  4. **지원도 평가**: 답변이 문서에 의해 뒷받침되는가? (Fully/Partially/No)
  5. **유용성 평가**: 답변이 질문에 유용한가? (1-5)
- **효과**: 더 신뢰할 수 있고 근거 기반의 응답 생성

**Self-RAG의 장점:**
- 불필요한 검색 방지
- 관련 없는 문서 필터링
- 근거 없는 hallucination 감소
- 응답 품질 자동 평가

**실행 결과:**
- 개념 설명이 출력됩니다.


In [ ]:
print("=" * 80)
print("[Self-RAG 개념]")
print("=" * 80)
print()
print("Self-RAG 프로세스:")
print()
print("1. 검색 필요성 평가")
print("   Q: '2+2는 얼마?' → A: 'No' (검색 불필요)")
print("   Q: 'Virtual memory란?' → A: 'Yes' (검색 필요)")
print()
print("2. 검색 수행 (필요한 경우)")
print("   → Top-K 문서 검색")
print()
print("3. 관련성 평가")
print("   각 문서: Relevant / Irrelevant")
print()
print("4. 응답 생성")
print("   관련 문서 기반으로 답변 생성")
print()
print("5. 지원도 평가")
print("   답변이 문서에 의해 뒷받침되는가?")
print("   → Fully supported / Partially / No support")
print()
print("6. 유용성 평가")
print("   답변이 질문에 얼마나 유용한가? (1-5)")
print()
print("=" * 80)
print("💡 핵심: 시스템이 스스로 품질을 평가하여 신뢰도 향상")
print("=" * 80)


---
## 4️⃣ Self-RAG 함수 구현

이 셀에서는 Ollama를 사용하여 Self-RAG의 각 평가 단계를 구현합니다.

**주요 내용:**
- 검색 필요성 평가 함수
- 관련성 평가 함수
- 응답 생성 함수
- 지원도 평가 함수
- 유용성 평가 함수
- 전체 Self-RAG 통합 함수

**실행 결과:**
- 각 평가 함수가 정의됩니다.


In [ ]:
def ollama_call(prompt: str, max_tokens: int = 50) -> str:
    """Ollama API 호출 헬퍼 함수"""
    payload = {
        "model": LLM_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": TEMPERATURE,
            "num_predict": max_tokens,
        }
    }
    try:
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=30)
        r.raise_for_status()
        return r.json().get("response", "").strip()
    except Exception as e:
        return f"[Error] {e}"

# 1. 검색 필요성 평가
def assess_retrieval_need(query: str) -> bool:
    """검색이 필요한지 평가"""
    prompt = f"""Does the following question require retrieval of external information to answer?
Output ONLY 'Yes' or 'No', nothing else.

Question: {query}

Answer (Yes/No):"""
    response = ollama_call(prompt, max_tokens=10)
    return 'yes' in response.lower()

# 2. 관련성 평가
def assess_relevance(query: str, document: str) -> bool:
    """문서가 쿼리와 관련있는지 평가"""
    prompt = f"""Is the following document relevant to answering the question?
Output ONLY 'Relevant' or 'Irrelevant', nothing else.

Question: {query}

Document: {document[:400]}

Answer (Relevant/Irrelevant):"""
    response = ollama_call(prompt, max_tokens=10)
    return 'relevant' in response.lower() and 'irrelevant' not in response.lower()

# 3. 응답 생성
def generate_response(query: str, context: str) -> str:
    """문서 기반 응답 생성"""
    prompt = f"""Answer the following question based on the provided context.
Be concise and accurate.

Context: {context}

Question: {query}

Answer:"""
    response = ollama_call(prompt, max_tokens=200)
    return response

# 4. 지원도 평가
def assess_support(response: str, context: str) -> str:
    """응답이 컨텍스트에 의해 뒷받침되는지 평가"""
    prompt = f"""Is the response fully supported by the context?
Output ONLY one of: 'Fully supported', 'Partially supported', or 'No support'.

Context: {context[:300]}

Response: {response}

Assessment:"""
    result = ollama_call(prompt, max_tokens=20)
    if 'fully' in result.lower():
        return 'Fully supported'
    elif 'partially' in result.lower():
        return 'Partially supported'
    else:
        return 'No support'

# 5. 유용성 평가
def assess_utility(query: str, response: str) -> int:
    """응답의 유용성을 1-5로 평가"""
    prompt = f"""Rate how useful this response is for answering the question.
Output ONLY a number from 1 to 5, nothing else.

Question: {query}

Response: {response}

Utility score (1-5):"""
    result = ollama_call(prompt, max_tokens=10)
    try:
        score = int(''.join(filter(str.isdigit, result))[:1] or "3")
        return max(1, min(5, score))
    except:
        return 3

print("✓ Self-RAG 평가 함수 정의 완료")


---
## 5️⃣ 데모: Self-RAG 전체 프로세스 실행

이 셀에서는 Self-RAG의 전체 프로세스를 실행하고 각 단계의 결과를 확인합니다.

**주요 내용:**
- 검색 필요성 평가
- 검색 수행 
- 관련성 평가
- 응답 생성
- 지원도 및 유용성 평가

**실행 결과:**
- 각 단계의 평가 결과와 최종 응답이 출력됩니다.


In [ ]:
def distance_to_similarity(distances: np.ndarray) -> np.ndarray:
    """L2 거리를 유사도로 변환"""
    return 1.0 - (distances / 2.0)

def search(query: str, k: int = TOP_K):
    """임베딩 기반 검색"""
    faiss.ParameterSpace().set_index_parameter(INDEX, "efSearch", EF_SEARCH)
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=NORMALIZE).astype('float32')
    D, I = INDEX.search(q_emb, k)
    scores = distance_to_similarity(D[0])
    return I[0], scores

def self_rag(query: str):
    """Self-RAG 전체 프로세스"""
    print("=" * 80)
    print(f"Query: {query}")
    print("=" * 80)
    
    # 1. 검색 필요성 평가
    print("\n[1단계: 검색 필요성 평가]")
    need_retrieval = assess_retrieval_need(query)
    print(f"검색 필요? {'Yes' if need_retrieval else 'No'}")
    
    if not need_retrieval:
        print("\n검색 없이 직접 답변 생성...")
        response = generate_response(query, "No specific context provided.")
        print(f"\n[최종 응답]\n{response}")
        return response
    
    # 2. 검색 수행
    print("\n[2단계: 검색 수행]")
    indices, scores = search(query, k=TOP_K)
    print(f"✓ {len(indices)}개 문서 검색 완료")
    
    # 3. 관련성 평가
    print("\n[3단계: 관련성 평가]")
    relevant_docs = []
    for i, idx in enumerate(indices):
        if idx < 0:
            continue
        chunk = CHUNKS[idx]
        is_relevant = assess_relevance(query, chunk['text'])
        print(f"  문서 {i+1}: {'Relevant' if is_relevant else 'Irrelevant'}")
        if is_relevant:
            relevant_docs.append(chunk)
    
    if not relevant_docs:
        print("  → 관련 문서 없음. 검색 없이 답변 생성...")
        response = generate_response(query, "No relevant context found.")
        print(f"\n[최종 응답]\n{response}")
        return response
    
    print(f"  → {len(relevant_docs)}개 관련 문서 발견")
    
    # 4. 응답 생성
    print("\n[4단계: 응답 생성]")
    context = "\n\n".join([doc['text'][:300] for doc in relevant_docs])
    response = generate_response(query, context)
    print(f"✓ 응답 생성 완료")
    
    # 5. 지원도 평가
    print("\n[5단계: 지원도 평가]")
    support = assess_support(response, context)
    print(f"지원도: {support}")
    
    # 6. 유용성 평가
    print("\n[6단계: 유용성 평가]")
    utility = assess_utility(query, response)
    print(f"유용성: {utility}/5")
    
    # 최종 출력
    print("\n" + "=" * 80)
    print("[최종 응답]")
    print("=" * 80)
    print(response)
    print("\n[평가 요약]")
    print(f"  - 관련 문서 수: {len(relevant_docs)}")
    print(f"  - 지원도: {support}")
    print(f"  - 유용성: {utility}/5")
    print("=" * 80)
    
    return response

# 데모 실행
TEST_QUERY = "How does the operating system handle virtual memory?"
self_rag(TEST_QUERY)


---
## 6️⃣ 추가 테스트: 검색이 불필요한 쿼리

이 셀에서는 검색이 불필요한 간단한 질문으로 Self-RAG를 테스트합니다.

**주요 내용:**
- 사실적 지식이 필요 없는 질문 테스트
- 검색 필요성 평가의 동작 확인

**실행 결과:**
- 검색 없이 직접 답변이 생성됩니다.


In [ ]:
# 간단한 질문 테스트
simple_query = "What is 2 + 2?"
print("\n\n")
self_rag(simple_query)

print("\n✅ Self-RAG 테스트 완료!")


---
## 7️⃣ 요약 및 다음 단계

### 학습 내용 요약

1. **Self-RAG 개념**: 다단계 자기 평가를 통한 신뢰도 높은 RAG
2. **평가 단계**: 검색 필요성 → 관련성 → 지원도 → 유용성
3. **효과**: Hallucination 감소, 근거 기반 응답, 품질 자동 평가
4. **구현**: Ollama를 활용한 로컬 LLM 기반 평가 시스템

### Self-RAG의 장점

- ✅ 불필요한 검색 방지 (효율성)
- ✅ 관련 없는 문서 필터링 (정확도)
- ✅ 근거 없는 hallucination 감소 (신뢰도)
- ✅ 응답 품질 자동 평가 (투명성)

### Self-RAG의 고려사항

- ⚠️  다단계 LLM 호출로 인한 높은 레이턴시
- ⚠️  평가의 정확도가 LLM 능력에 의존
- ⚠️  간단한 질문에는 오버헤드

### 다음 단계

- **Chapter 7.5**: 모든 기법 통합 (CCH + HyDE + Reranking + Self-RAG)
  - CCH로 임베딩된 인덱스 사용
  - HyDE로 쿼리 확장
  - Reranking으로 문서 재정렬
  - Self-RAG로 전체 품질 관리

### 주요 함수

- `assess_retrieval_need(query)`: 검색 필요성 평가
- `assess_relevance(query, document)`: 문서 관련성 평가
- `generate_response(query, context)`: 응답 생성
- `assess_support(response, context)`: 지원도 평가
- `assess_utility(query, response)`: 유용성 평가
- `self_rag(query)`: 전체 Self-RAG 프로세스
